In [1]:
import jax 
import jax.numpy as jnp
import numpy as np 
import pandas as pd
from cluster import BasicClusteringRecommender, KMeansRecommender

In [2]:
train_df = pd.read_csv('../preprocessed_dataset/train_dataset.csv')
test_df = pd.read_csv('../preprocessed_dataset/test_dataset.csv')

In [3]:
train_df.head()

,Unnamed: 0.1,Unnamed: 0,userID,review_count,songID,rating,user_id_idx,item_id_idx
0,0,966383,5226,10,82304,5,1115,12236
1,1,753438,23119,10,61708,5,4880,9184
2,2,1156841,7835,10,122957,5,1667,18346
3,3,503221,16895,10,110494,5,3568,16467
4,4,415592,15695,10,57161,5,3321,8548


In [4]:
train_df.describe()

,Unnamed: 0.1,Unnamed: 0,userID,review_count,songID,rating,user_id_idx,item_id_idx
count,40000.00000,4.000000e+04,40000.000000,40000.0,40000.000000,40000.000000,40000.000000,40000.000000
mean,19999.50000,6.657422e+05,11804.231675,10.0,68053.038100,4.549575,2497.230375,10150.078850
std,11547.14972,3.504432e+05,6830.278556,0.0,39515.208908,0.696171,1442.006203,5875.358351
min,0.00000,0.000000e+00,0.000000,10.0,2.000000,3.000000,0.000000,0.000000
25%,9999.75000,3.801635e+05,5835.000000,10.0,33848.750000,4.000000,1251.000000,5088.750000
50%,19999.50000,7.344685e+05,11837.000000,10.0,67838.500000,5.000000,2495.000000,10079.500000
75%,29999.25000,9.247522e+05,17732.000000,10.0,102227.000000,5.000000,3746.000000,15246.000000
max,39999.00000,1.315059e+06,23680.000000,10.0,136729.000000,5.000000,4999.000000,20360.000000


In [5]:
test_df.describe()

,Unnamed: 0,userID,review_count,songID,rating,user_id_idx,item_id_idx
count,2466.000000,2466.000000,2466.0,2466.000000,2466.000000,2466.000000,2466.000000
mean,3307.095702,11858.433496,10.0,67045.714517,4.558800,2508.562855,9999.652068
std,1912.952314,6858.378833,0.0,39385.201394,0.685987,1447.535411,5855.695673
min,0.000000,90.000000,10.0,60.000000,3.000000,17.000000,10.000000
25%,1608.000000,6005.000000,10.0,33557.750000,4.000000,1284.500000,5045.000000
50%,3355.500000,11871.500000,10.0,66685.000000,5.000000,2504.000000,9901.000000
75%,4928.250000,17832.750000,10.0,99568.750000,5.000000,3764.250000,14847.250000
max,6603.000000,23668.000000,10.0,136507.000000,5.000000,4997.000000,20337.000000


In [6]:
train_df = train_df[['rating', 'user_id_idx', 'item_id_idx']]
test_df = test_df[['rating', 'user_id_idx', 'item_id_idx']]

In [7]:
train_df.rename(columns={'user_id_idx': 'userID', 'item_id_idx': 'songID'}, inplace=True)
test_df.rename(columns={'user_id_idx': 'userID', 'item_id_idx': 'songID'}, inplace=True)

In [8]:
test_df.shape

(2466, 3)

In [9]:
test_df.userID.nunique()

736

In [10]:
def eval_model(model, test_df):
    tp = 0
    users = test_df.userID.unique()
    num_recommendation = 3
    total_num_recommendations = num_recommendation * len(users)
    num_test_songs = len(test_df)

    for user in users:
        recommendations = model.predict(user, num_recommendation)
        tp += len(test_df[test_df.userID == user & np.isin(test_df.songID, recommendations)])

    precision = tp / total_num_recommendations
    recall = tp / num_test_songs 
    f1 = 0 if tp == 0 else 2 / (1/recall + 1/precision)

    return precision, recall, f1

**Evaluate Basic Clustering Recommender**

In [21]:
base_recommender = BasicClusteringRecommender(n_neighbor=50)

base_recommender.fit(train_df)

eval_model(base_recommender, test_df) # this may take quite long
# We can get:
# precision = 0.0005
# recall = 0.0004
# f1 = 0.0004

**Evaluate K-Means Recommdender with Minibatch**

In [14]:
minibatch_kmeans_recommender = KMeansRecommender(n_neighbor=50, use_minibatch=True)

minibatch_kmeans_recommender.fit(train_df)

eval_model(minibatch_kmeans_recommender, test_df) # this may take quite long
# We can get:
# precision = 0.0005
# recall = 0.0004
# f1 = 0.0004

**Evaluate K-Means Recommender without Minibatch**

In [17]:
regular_kmeans_recommender = KMeansRecommender(n_neighbor=50, use_minibatch=False)

regular_kmeans_recommender.fit(train_df)

eval_model(regular_kmeans_recommender, test_df) # this may take quite long
# We can get:
# precision = 0.0005
# recall = 0.0004
# f1 = 0.0004